In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..

/home/imkbsz/workspace/eeg_analysis


In [2]:
import os
import numpy as np
import wandb
import matplotlib.pyplot as plt

In [3]:
api = wandb.Api()
runs = api.runs('caueeg-mae')

pre_name_to_mask_ratio = {}
for run in runs:
    pre_name_to_mask_ratio[run.name] = run.config["mask_ratio"]
print(pre_name_to_mask_ratio)

{'eg6s5fay': 0.75, 'r3db85gm': 0.75, '9rrhec00': 0.75, '5xlos421': 0.5, 'rhqfowxp': 0.5, '255ugwah': 0.5, 'cmru56bi': 0.5, 'nfbtrm5s': 0.2, '3f90hsjz': 0.5, 'p25u3bun': 0.5, '0ei2cjne': 0.5, '4xb6ohzp': 0.25, '0473gnjs': 0.5, 'ah6loko1': 0.5, '7uf9ofni': 0.5, 'voosm1bm': 0.5, 'klo0kw08': 0.5, 'wxaaytah': 0.5, 'fvuepvb9': 0.25, '4ui9h39h': 0.75, 'wftcdahj': 0.5, '2ew55ua4': 0.5, 'dch78dwg': 0.5}


In [4]:
api = wandb.Api()
runs = api.runs('tuab-mae')

# pre_name_to_mask_ratio = {}
for run in runs:
    pre_name_to_mask_ratio[run.name] = run.config["mask_ratio"]
print(pre_name_to_mask_ratio)

{'eg6s5fay': 0.75, 'r3db85gm': 0.75, '9rrhec00': 0.75, '5xlos421': 0.5, 'rhqfowxp': 0.5, '255ugwah': 0.5, 'cmru56bi': 0.5, 'nfbtrm5s': 0.2, '3f90hsjz': 0.5, 'p25u3bun': 0.5, '0ei2cjne': 0.5, '4xb6ohzp': 0.25, '0473gnjs': 0.5, 'ah6loko1': 0.5, '7uf9ofni': 0.5, 'voosm1bm': 0.5, 'klo0kw08': 0.5, 'wxaaytah': 0.5, 'fvuepvb9': 0.25, '4ui9h39h': 0.75, 'wftcdahj': 0.5, '2ew55ua4': 0.5, 'dch78dwg': 0.5, 'e57t1ac8': 0.75, 'nyb2t0p2': 0.75, '40o98jml': 0.75, 'vt7lwk2m': 0.75, 'jwf6efps': 0.75, '7akfmk44': 0.5}


In [10]:
api = wandb.Api()
runs = api.runs('caueeg-mae-artifact-finetune')

for run in runs:
    if run.config.get("_target_", "").startswith("models.mae_artifact") and "descending" in run.config:
        descending = run.config.pop("descending")
        art_patch_usage = {}
        art_patch_usage["type"] = "drop_low" if descending else "drop_high"
        art_patch_usage["value"] = run.config["mask_ratio"]
        run.config["mask_ratio"] = pre_name_to_mask_ratio[run.config["pre_model"]]
        run.config["art_patch_usage"] = art_patch_usage
        run.config["art_out_activation"] = "none"
        run.config["art_loss_type"] = "mse"
        run.config.update()
        run.update()

In [11]:
api = wandb.Api()
runs = api.runs('caueeg-abnormal-mae-artifact-finetune')

for run in runs:
    if run.config.get("_target_", "").startswith("models.mae_artifact") and "descending" in run.config:
        descending = run.config.pop("descending")
        art_patch_usage = {}
        art_patch_usage["type"] = "drop_low" if descending else "drop_high"
        art_patch_usage["value"] = run.config["mask_ratio"]
        run.config["mask_ratio"] = pre_name_to_mask_ratio[run.config["pre_model"]]
        run.config["art_patch_usage"] = art_patch_usage
        run.config["art_out_activation"] = "none"
        run.config["art_loss_type"] = "mse"
        run.config.update()
        run.update()

In [12]:
api = wandb.Api()
runs = api.runs('tuab-mae-artifact-finetune')

for run in runs:
    if run.config.get("_target_", "").startswith("models.mae_artifact") and "descending" in run.config:
        descending = run.config.pop("descending")
        art_patch_usage = {}
        art_patch_usage["type"] = "drop_low" if descending else "drop_high"
        art_patch_usage["value"] = run.config["mask_ratio"]
        run.config["mask_ratio"] = pre_name_to_mask_ratio[run.config["pre_model"]]
        run.config["art_patch_usage"] = art_patch_usage
        run.config["art_out_activation"] = "none"
        run.config["art_loss_type"] = "mse"
        run.config.update()
        run.update()

In [ ]:
import glob
import torch
from tqdm.auto import tqdm
import shutil
import os

print('PyTorch version:', torch.__version__)
device = 'cuda:0'
device = torch.device(device if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.')

for f in tqdm(glob.glob(r"./local/checkpoint/*/checkpoint.pt")):
    # load pretrained configurations
    try:
        ckpt = torch.load(f, map_location=device)
        config = ckpt['config']
        if config["_target_"].startswith("models.mae_artifact") and "descending" in config:
            descending = config.pop("descending")
            art_patch_usage = {}
            art_patch_usage["type"] = "drop_low" if descending else "drop_high"
            art_patch_usage["value"] = run.config["mask_ratio"]
            config["mask_ratio"] = pre_name_to_mask_ratio[config["pre_model"]]
            config["art_patch_usage"] = art_patch_usage
            config["art_out_activation"] = "none"
            config["art_loss_type"] = "mse"
            ckpt["config"] = config
            torch.save(ckpt, f)
                        
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {f}')

In [ ]:
# run_names = ['mwwzp6b1', 'q3fgkjoc', '3xg5i2z2', 'ccstq34j', 'emknf2qj',
#              'hysvvul2', 'va9swnif', 'e7c3x7iz', 'uidjldyf', '353eu3zg',]

# api = wandb.Api()

# for run_name in run_names:
#     run = api.run(f"ipis-mjkim/caueeg-dementia/{run_name}")
    
#     run.config['distil_type'] = 'hard'
#     run.config['distil_alpha'] = run.config['distillation_ratio']
#     del run.config['distillation_ratio']
#     run.config['distil_tau'] = 1.0
#     run.config['distil_teacher'] = run.config['teacher']['name']
#     run.config['distil_teacher_criterion'] = run.config['teacher']['criterion']
#     del run.config['teacher']
    
#     run.config.update()
#     run.update()

In [ ]:
# api = wandb.Api()
# wandb.init(project='caueeg-task1', id='atbhqdgg', resume='must')
# wandb.run.finish()

In [ ]:
# api = wandb.Api()
# runs = api.runs('ipis-mjkim/caueeg-task1')

# for run in runs:
#     # print(run, run.config['model'])
#     # if run.name == '1tdyketc':
#     if run.name == '18lkfaaw':
#     # if run.state == "finished":
#         print(run.name)

#         confusion = np.array(run.summary['Confusion Matrix (Array)'])
#         n_classes = confusion.shape[0]

#         accuracy = np.zeros((n_classes,))
#         sensitivity = np.zeros((n_classes,))
#         specificity = np.zeros((n_classes,))
#         precision = np.zeros((n_classes,))
#         recall = np.zeros((n_classes,))
#         f1_score = np.zeros((n_classes,))

#         for c in range(n_classes):
#             tp = confusion[c, c]
#             fn = confusion[c].sum() - tp
#             fp = confusion[:, c].sum() - tp
#             tn = confusion.sum() - tp - fn - fp

#             accuracy[c] = (tp + tn) / (tp + fn + fp + tn)
#             sensitivity[c] = tp / (tp + fn)
#             specificity[c] = tn / (fp + tn)
#             precision[c] = tp / (tp + fp)
#             recall[c] = tp / (tp + fn)

#         f1_score = 2 * precision * recall / (precision + recall)
        
#         print(run.summary['Test Accuracy'])
#         print('accuracy:', accuracy)
#         print('sensitivity:', sensitivity)
#         print('specificity:', specificity)
#         print('precision:', precision)
#         print('recall:', recall)
#         print('f1_score:', f1_score)
#         break

In [ ]:
# api = wandb.Api()
# runs = api.runs('ipis-mjkim/eeg-analysis')

# for run in runs:
#     print(run)
#     debug_table_serial = run.summary['Test Debug Table/Serial']
#     debug_table_edf = run.summary['Test Debug Table/EDF']
#     debug_table_pred = run.summary['Test Debug Table/Pred']
#     debug_table_gt = run.summary['Test Debug Table/GT']
    
#     fig = plt.figure(num=1, clear=True, figsize=(20.0, 4.0), constrained_layout=True)
#     ax = fig.add_subplot(1, 1, 1)

#     total_error, total_count = (0, 0)

#     for edf in np.unique(debug_table_edf):
#         indices = [i for i, x in enumerate(debug_table_edf) if x == edf]

#         err, cnt = (0, 0)
#         for i in indices:
#             cnt += sum(debug_table_pred[i])
#             err += sum(debug_table_pred[i]) - debug_table_pred[i][debug_table_gt[i]]

#         total_error += err
#         total_count += cnt

#         ax.bar(edf, err / cnt, color=['g', 'b', 'r'][debug_table_gt[i]])

#     ax.set_title(f'Test Debug Table (Acc. {1.0 - total_error / total_count: .2f}%)', fontsize=18)
#     ax.set_ylim(0.0, 1.0)
#     plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)

#     # run.summary['Test Debug Table (Image)'] = wandb.Image(plt)
#     # run.summary.update()
#     print(type(wandb.Image(plt)))
#     run.summary.update({'Test Debug Table (Image)': wandb.Image(plt)})
#     fig.clear()
#     plt.close(fig)

In [ ]:
# api = wandb.Api()
# run = api.run("ipis-mjkim/eeg-analysis/1nhn425l")
# print(run)

# run.summary.update()

In [ ]:
# api = wandb.Api()
# runs = api.runs('ipis-mjkim/eeg-analysis')

# for run in runs:
#     print(run)
#     if 'Age' in run.config:
#         del run.config['Age']
#         run.config.update()
#         run.update()        
    
#     if 'no-age' not in run.config['model']:
#         run.config['use_age'] = 'fc'
#         run.config.update()
#         run.update()
#     else:
#         run.config['use_age'] = None
#         run.config.update()
#         run.update()        

In [ ]:
# api = wandb.Api()
# runs = api.runs('ipis-mjkim/eeg-analysis')

# for run in runs:
#     print(run)
#     debug_table_serial = run.summary['Test Debug Table/Serial']
#     debug_table_edf = run.summary['Test Debug Table/EDF']
#     debug_table_pred = run.summary['Test Debug Table/Pred']
#     debug_table_gt = run.summary['Test Debug Table/GT']
    
#     fig = plt.figure(num=1, clear=True, figsize=(20.0, 4.0), constrained_layout=True)
#     ax = fig.add_subplot(1, 1, 1)

#     total_error, total_count = (0, 0)

#     for edf in np.unique(debug_table_edf):
#         indices = [i for i, x in enumerate(debug_table_edf) if x == edf]

#         err, cnt = (0, 0)
#         for i in indices:
#             cnt += sum(debug_table_pred[i])
#             err += sum(debug_table_pred[i]) - debug_table_pred[i][debug_table_gt[i]]

#         total_error += err
#         total_count += cnt

#         ax.bar(edf, err / cnt, color=['g', 'b', 'r'][debug_table_gt[i]])

#     ax.set_title(f'Test Debug Table (Acc. {1.0 - total_error / total_count: .2f}%)', fontsize=18)
#     ax.set_ylim(0.0, 1.0)
#     plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)

#     # run.summary['Test Debug Table (Image)'] = wandb.Image(plt)
#     # run.summary.update()
#     print(type(wandb.Image(plt)))
#     run.summary.update({'Test Debug Table (Image)': wandb.Image(plt)})
#     fig.clear()
#     plt.close(fig)

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..


In [ ]:
# import glob
# import torch
# from tqdm.auto import tqdm
# from run_train import generate_model

# print('PyTorch version:', torch.__version__)
# device = 'cuda:0'
# device = torch.device(device if torch.cuda.is_available() else 'cpu')

# if torch.cuda.is_available(): print('cuda is available.')
# else: print('cuda is unavailable.')

# for f in tqdm(glob.glob(r"D:\GitHub\eeg_analysis\local\checkpoint\1udvls4y\checkpoint.pt")):
#     # load pretrained configurations
#     try:
#         ckpt = torch.load(f, map_location=device)
#         config = ckpt['config']
#         if "_target_" in config.keys() and 'mae' in config["_target_"] and 'mae_1d' not in config["_target_"]:
#             config["_target_"] = config["_target_"].replace('mae', 'mae_1d')
#             ckpt['config'] = config
#             torch.save(ckpt, f)
            
#     except Exception as e:
#         print(e)
#         print(f'- checkpoint cannot be opened: {f}')

In [ ]:
# import glob
# import torch
# from tqdm.auto import tqdm
# import shutil
# import os

# print('PyTorch version:', torch.__version__)
# device = 'cuda:0'
# device = torch.device(device if torch.cuda.is_available() else 'cpu')

# if torch.cuda.is_available(): print('cuda is available.')
# else: print('cuda is unavailable.')

# for f in tqdm(glob.glob(r"D:\GitHub\eeg_analysis\local\checkpoint\*\checkpoint.pt")):
#     # load pretrained configurations
#     try:
#         ckpt = torch.load(f, map_location=device)
#         config = ckpt['config']
#         if config["dataset_name"] == "tuab":
#             print(os.path.dirname(f))
#             shutil.rmtree(os.path.dirname(f))
                        
#     except Exception as e:
#         print(e)
#         print(f'- checkpoint cannot be opened: {f}')